# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f71b8cdb2e0>
├── 'a' --> tensor([[ 0.4121, -0.8137, -1.0268],
│                   [ 1.3375, -0.7847,  1.0925]])
└── 'x' --> <FastTreeValue 0x7f71b8cdb850>
    └── 'c' --> tensor([[-0.0350, -0.1335, -0.3865, -0.2587],
                        [-1.0456, -0.7926, -1.2495, -0.3003],
                        [ 0.5578, -0.6303, -1.4303, -1.7862]])

In [4]:
t.a

tensor([[ 0.4121, -0.8137, -1.0268],
        [ 1.3375, -0.7847,  1.0925]])

In [5]:
%timeit t.a

79.1 ns ± 1.47 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f71b8cdb2e0>
├── 'a' --> tensor([[ 2.0080, -0.2371,  0.1596],
│                   [ 0.2532,  1.6639, -0.5951]])
└── 'x' --> <FastTreeValue 0x7f71b8cdb850>
    └── 'c' --> tensor([[-0.0350, -0.1335, -0.3865, -0.2587],
                        [-1.0456, -0.7926, -1.2495, -0.3003],
                        [ 0.5578, -0.6303, -1.4303, -1.7862]])

In [7]:
%timeit t.a = new_value

81.7 ns ± 0.714 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.4121, -0.8137, -1.0268],
               [ 1.3375, -0.7847,  1.0925]]),
    x: Batch(
           c: tensor([[-0.0350, -0.1335, -0.3865, -0.2587],
                      [-1.0456, -0.7926, -1.2495, -0.3003],
                      [ 0.5578, -0.6303, -1.4303, -1.7862]]),
       ),
)

In [10]:
b.a

tensor([[ 0.4121, -0.8137, -1.0268],
        [ 1.3375, -0.7847,  1.0925]])

In [11]:
%timeit b.a

71.1 ns ± 0.515 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 2.3102,  0.1533, -0.5103],
               [-1.9165,  0.6871,  0.8805]]),
    x: Batch(
           c: tensor([[-0.0350, -0.1335, -0.3865, -0.2587],
                      [-1.0456, -0.7926, -1.2495, -0.3003],
                      [ 0.5578, -0.6303, -1.4303, -1.7862]]),
       ),
)

In [13]:
%timeit b.a = new_value

661 ns ± 5.62 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.1 µs ± 13.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

14.3 µs ± 123 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

284 µs ± 9.08 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

273 µs ± 10.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f70e26c1df0>
├── 'a' --> tensor([[[ 0.4121, -0.8137, -1.0268],
│                    [ 1.3375, -0.7847,  1.0925]],
│           
│                   [[ 0.4121, -0.8137, -1.0268],
│                    [ 1.3375, -0.7847,  1.0925]],
│           
│                   [[ 0.4121, -0.8137, -1.0268],
│                    [ 1.3375, -0.7847,  1.0925]],
│           
│                   [[ 0.4121, -0.8137, -1.0268],
│                    [ 1.3375, -0.7847,  1.0925]],
│           
│                   [[ 0.4121, -0.8137, -1.0268],
│                    [ 1.3375, -0.7847,  1.0925]],
│           
│                   [[ 0.4121, -0.8137, -1.0268],
│                    [ 1.3375, -0.7847,  1.0925]],
│           
│                   [[ 0.4121, -0.8137, -1.0268],
│                    [ 1.3375, -0.7847,  1.0925]],
│           
│                   [[ 0.4121, -0.8137, -1.0268],
│                    [ 1.3375, -0.7847,  1.0925]]])
└── 'x' --> <FastTreeValue 0x7f71bcd7c3d0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

47.3 µs ± 374 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f71b8d48fa0>
├── 'a' --> tensor([[ 0.4121, -0.8137, -1.0268],
│                   [ 1.3375, -0.7847,  1.0925],
│                   [ 0.4121, -0.8137, -1.0268],
│                   [ 1.3375, -0.7847,  1.0925],
│                   [ 0.4121, -0.8137, -1.0268],
│                   [ 1.3375, -0.7847,  1.0925],
│                   [ 0.4121, -0.8137, -1.0268],
│                   [ 1.3375, -0.7847,  1.0925],
│                   [ 0.4121, -0.8137, -1.0268],
│                   [ 1.3375, -0.7847,  1.0925],
│                   [ 0.4121, -0.8137, -1.0268],
│                   [ 1.3375, -0.7847,  1.0925],
│                   [ 0.4121, -0.8137, -1.0268],
│                   [ 1.3375, -0.7847,  1.0925],
│                   [ 0.4121, -0.8137, -1.0268],
│                   [ 1.3375, -0.7847,  1.0925]])
└── 'x' --> <FastTreeValue 0x7f70e295c640>
    └── 'c' --> tensor([[-0.0350, -0.1335, -0.3865, -0.2587],
                        [-1.0456, -0.7926, -1.2495, -0.3003],
                 

In [23]:
%timeit t_cat(trees)

43.8 µs ± 777 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

93.8 µs ± 604 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.0350, -0.1335, -0.3865, -0.2587],
                       [-1.0456, -0.7926, -1.2495, -0.3003],
                       [ 0.5578, -0.6303, -1.4303, -1.7862]],
              
                      [[-0.0350, -0.1335, -0.3865, -0.2587],
                       [-1.0456, -0.7926, -1.2495, -0.3003],
                       [ 0.5578, -0.6303, -1.4303, -1.7862]],
              
                      [[-0.0350, -0.1335, -0.3865, -0.2587],
                       [-1.0456, -0.7926, -1.2495, -0.3003],
                       [ 0.5578, -0.6303, -1.4303, -1.7862]],
              
                      [[-0.0350, -0.1335, -0.3865, -0.2587],
                       [-1.0456, -0.7926, -1.2495, -0.3003],
                       [ 0.5578, -0.6303, -1.4303, -1.7862]],
              
                      [[-0.0350, -0.1335, -0.3865, -0.2587],
                       [-1.0456, -0.7926, -1.2495, -0.3003],
                       [ 0.5578, -0.6303, -1.4303, -1.7862]],

In [26]:
%timeit Batch.stack(batches)

114 µs ± 985 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.0350, -0.1335, -0.3865, -0.2587],
                      [-1.0456, -0.7926, -1.2495, -0.3003],
                      [ 0.5578, -0.6303, -1.4303, -1.7862],
                      [-0.0350, -0.1335, -0.3865, -0.2587],
                      [-1.0456, -0.7926, -1.2495, -0.3003],
                      [ 0.5578, -0.6303, -1.4303, -1.7862],
                      [-0.0350, -0.1335, -0.3865, -0.2587],
                      [-1.0456, -0.7926, -1.2495, -0.3003],
                      [ 0.5578, -0.6303, -1.4303, -1.7862],
                      [-0.0350, -0.1335, -0.3865, -0.2587],
                      [-1.0456, -0.7926, -1.2495, -0.3003],
                      [ 0.5578, -0.6303, -1.4303, -1.7862],
                      [-0.0350, -0.1335, -0.3865, -0.2587],
                      [-1.0456, -0.7926, -1.2495, -0.3003],
                      [ 0.5578, -0.6303, -1.4303, -1.7862],
                      [-0.0350, -0.1335, -0.3865, -0.2587],
                   

In [28]:
%timeit Batch.cat(batches)

201 µs ± 1.87 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

617 µs ± 22.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
